In [8]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import os,PIL,pathlib
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

#设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
data_dir = './data/bird_photos'
data_dir = pathlib.Path(data_dir)

In [3]:
data_dir = './data/bird_photos'
data_transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor()
])

In [4]:
total_data = datasets.ImageFolder(data_dir,transform=data_transform)
total_data

Dataset ImageFolder
    Number of datapoints: 566
    Root location: ./data/bird_photos
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=bilinear)
               ToTensor()
           )

In [5]:
train_size = int(0.8*len(total_data))
test_size = len(total_data) - train_size
train_data,test_data = torch.utils.data.random_split(total_data,[train_size,test_size])

In [23]:
batch_size = 8
train_dl = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_data,batch_size=batch_size)

In [10]:
class inception_block(nn.Module):
    def __init__(self,in_channels,ch1x1,ch3x3red,ch3x3,ch5x5red,ch5x5,pool_proj):
        super().__init__()
        # 1x1 conv branch
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels,ch1x1,kernel_size=1),
            nn.BatchNorm2d(ch1x1),
            nn.ReLU(inplace=True)
        )
        # 1x1 conv -> 3x3 conv branch
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels,ch3x3red,kernel_size=1),
            nn.BatchNorm2d(ch3x3red),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3red,ch3x3,kernel_size=3,padding=1),
            nn.BatchNorm2d(ch3x3),
            nn.ReLU(inplace=True)
        )
        # 1x1 conv -> 5x5 conv branch
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels,ch5x5red,kernel_size=1),
            nn.BatchNorm2d(ch5x5red),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5red,ch5x5,kernel_size=3,padding=1),
            nn.BatchNorm2d(ch5x5),
            nn.ReLU(inplace=True)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(in_channels,pool_proj,kernel_size=1),
            nn.BatchNorm2d(pool_proj),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        branch1_output = self.branch1(x)
        branch2_output = self.branch2(x)
        branch3_output = self.branch3(x)
        branch4_output = self.branch4(x)
        
        outputs = [branch1_output,branch2_output,branch3_output,branch4_output]
        return torch.cat(outputs,1)

In [26]:
class InceptionV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionV1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception3a = inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2)
        self.inception4a = inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(2, stride=2)
        self.inception5a = inception_block(832, 256, 160, 320, 32, 128, 128)

        self.inception5b=nn.Sequential(
        inception_block(832, 384, 192, 384, 48, 128, 128),
        nn.AvgPool2d(kernel_size=7,stride=1,padding=0),
        nn.Dropout(0.4)
        )
        # 全连接网络层，用于分类
        self.classifier = nn.Sequential(
        nn.Linear(in_features=1024, out_features=1024),
        nn.ReLU(),
        nn.Linear(in_features=1024, out_features=num_classes),
        nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x;

In [30]:
model = InceptionV1(4).to(device)

In [31]:
import torchsummary
torchsummary.summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
         MaxPool2d-2           [-1, 64, 56, 56]               0
            Conv2d-3           [-1, 64, 58, 58]           4,160
            Conv2d-4          [-1, 192, 58, 58]         110,784
         MaxPool2d-5          [-1, 192, 29, 29]               0
            Conv2d-6           [-1, 64, 29, 29]          12,352
       BatchNorm2d-7           [-1, 64, 29, 29]             128
              ReLU-8           [-1, 64, 29, 29]               0
            Conv2d-9           [-1, 96, 29, 29]          18,528
      BatchNorm2d-10           [-1, 96, 29, 29]             192
             ReLU-11           [-1, 96, 29, 29]               0
           Conv2d-12          [-1, 128, 29, 29]         110,720
      BatchNorm2d-13          [-1, 128, 29, 29]             256
             ReLU-14          [-1, 128,

In [32]:
# 训练循环
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 训练集的大小
    num_batches = len(dataloader)   # 批次数目, (size/batch_size，向上取整)

    train_loss, train_acc = 0, 0  # 初始化训练损失和正确率
    
    for X, y in dataloader:  # 获取图片及其标签
        X, y = X.to(device), y.to(device)
        
        # 计算预测误差
        pred = model(X)          # 网络输出
        loss = loss_fn(pred, y)  # 计算网络输出和真实值之间的差距，targets为真实值，计算二者差值即为损失
        
        # 反向传播
        optimizer.zero_grad()  # grad属性归零
        loss.backward()        # 反向传播
        optimizer.step()       # 每一步自动更新
        
        # 记录acc与loss
        train_acc  += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss += loss.item()
            
    train_acc  /= size
    train_loss /= num_batches

    return train_acc, train_loss

In [33]:
def test (dataloader, model, loss_fn):
    size        = len(dataloader.dataset)  # 测试集的大小
    num_batches = len(dataloader)          # 批次数目, (size/batch_size，向上取整)
    test_loss, test_acc = 0, 0
    
    # 当不进行训练时，停止梯度更新，节省计算内存消耗
    with torch.no_grad():
        for imgs, target in dataloader:
            imgs, target = imgs.to(device), target.to(device)
            
            # 计算loss
            target_pred = model(imgs)
            loss        = loss_fn(target_pred, target)
            
            test_loss += loss.item()
            test_acc  += (target_pred.argmax(1) == target).type(torch.float).sum().item()

    test_acc  /= size
    test_loss /= num_batches

    return test_acc, test_loss

In [34]:
import copy

optimizer  = torch.optim.Adam(model.parameters(), lr= 1e-4)
loss_fn    = nn.CrossEntropyLoss() # 创建损失函数

epochs     = 10

train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

best_acc = 0    # 设置一个最佳准确率，作为最佳模型的判别指标

for epoch in range(epochs):
    
    model.train()
    epoch_train_acc, epoch_train_loss = train(train_dl, model, loss_fn, optimizer)
    
    model.eval()
    epoch_test_acc, epoch_test_loss = test(test_dl, model, loss_fn)
    
    # 保存最佳模型到 best_model
    if epoch_test_acc > best_acc:
        best_acc   = epoch_test_acc
        best_model = copy.deepcopy(model)
    
    train_acc.append(epoch_train_acc)
    train_loss.append(epoch_train_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
    
    # 获取当前的学习率
    lr = optimizer.state_dict()['param_groups'][0]['lr']
    
    template = ('Epoch:{:2d}, Train_acc:{:.1f}%, Train_loss:{:.3f}, Test_acc:{:.1f}%, Test_loss:{:.3f}, Lr:{:.2E}')
    print(template.format(epoch+1, epoch_train_acc*100, epoch_train_loss, 
                          epoch_test_acc*100, epoch_test_loss, lr))
    

print('Done')

Epoch: 1, Train_acc:54.0%, Train_loss:1.223, Test_acc:16.7%, Test_loss:1.547, Lr:1.00E-04
Epoch: 2, Train_acc:70.1%, Train_loss:1.047, Test_acc:72.8%, Test_loss:1.052, Lr:1.00E-04
Epoch: 3, Train_acc:74.6%, Train_loss:1.003, Test_acc:68.4%, Test_loss:1.037, Lr:1.00E-04
Epoch: 4, Train_acc:77.9%, Train_loss:0.971, Test_acc:77.2%, Test_loss:0.987, Lr:1.00E-04
Epoch: 5, Train_acc:75.0%, Train_loss:0.988, Test_acc:77.2%, Test_loss:0.958, Lr:1.00E-04
Epoch: 6, Train_acc:82.3%, Train_loss:0.924, Test_acc:86.8%, Test_loss:0.885, Lr:1.00E-04
Epoch: 7, Train_acc:83.4%, Train_loss:0.906, Test_acc:78.1%, Test_loss:0.976, Lr:1.00E-04
Epoch: 8, Train_acc:84.1%, Train_loss:0.906, Test_acc:84.2%, Test_loss:0.884, Lr:1.00E-04
Epoch: 9, Train_acc:86.5%, Train_loss:0.883, Test_acc:83.3%, Test_loss:0.892, Lr:1.00E-04
Epoch:10, Train_acc:86.1%, Train_loss:0.884, Test_acc:86.8%, Test_loss:0.877, Lr:1.00E-04
Done
